# Импорт модулей

In [3]:
import json
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import log_loss

# Загрузка данных

Тренировочные данные состоят из полей 'text_id', 'text' и 11 полей с таргетами. Не стоит забывать, что может быть больше одной болезни для каждого случая.

Тестовые же данные содержат поля 'text_id' и 'text'.

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train.head(2)

,text_id,text,эймериоз,абсцесс,диспепсия молодняка,остертагиоз,мастит,инфекционный ринотрахеит,отёк вымени,тенденит,сибирская язва,лишай,другое
0,0,"Корова, видимо вставая, раздавила себе сосок. ...",0,0,0,0,1,0,0,0,0,0,0
1,1,Корове 8 лет! Месяц назад промеж четвертей вым...,0,0,0,0,0,0,0,0,1,1,1


In [7]:
test.head(2)

,text_id,text
0,294,Понос у месячных телят. Подскажите методы и сп...
1,295,"Понос у телят, чем лечить? \nЧем можно вылечит..."


# Базовая модель

В качестве базового решения используется CatBoostClassifier, поддерживающий текстовые фичи. Помимо этого задача является мультилейбл классификацией, поэтому модель обернута в OneVsRestClassifier.

**Делим данные на тренировочную и валидационную выборку**

In [8]:
X_train, X_val, y_train, y_val = train_test_split(pd.DataFrame(train['text']), train[train.columns[2:]], test_size=0.2)

**Обучаем модель**

In [9]:
model = OneVsRestClassifier(estimator=CatBoostClassifier(iterations = 100, text_features=['text'], 
                                                         verbose = 50, allow_writing_files=False))
model.fit(X_train, y_train)

Learning rate set to 0.045854
0:	learn: 0.6747707	total: 162ms	remaining: 16.1s
50:	learn: 0.3458487	total: 1.46s	remaining: 1.41s
99:	learn: 0.2733018	total: 2.58s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6751378	total: 13.8ms	remaining: 1.36s
50:	learn: 0.3927736	total: 1.32s	remaining: 1.27s
99:	learn: 0.3389888	total: 2.58s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6741963	total: 25.7ms	remaining: 2.55s
50:	learn: 0.3692726	total: 1.29s	remaining: 1.24s
99:	learn: 0.3088863	total: 2.53s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6721095	total: 27.8ms	remaining: 2.75s
50:	learn: 0.3370063	total: 1.44s	remaining: 1.39s
99:	learn: 0.2708549	total: 2.69s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6703666	total: 23.6ms	remaining: 2.33s
50:	learn: 0.3134060	total: 1.37s	remaining: 1.31s
99:	learn: 0.2479107	total: 2.6s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6698878	total: 27.6ms	remaining: 2.73s
50:	learn: 0.313

OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x0000022E6741EDC0>)

# Считаем метрику

В качестве "gt" (ground truth) функция принимает на вход датафрейм/массив из 10 столбцов (все классы, кроме "другое"). Предсказанные значения – "pr", также должны быть либо в виде датафрейма, либо в виде массива.

In [10]:
def log_loss_score(gt, pr):
    
    log_loss_ = 0
    
    gt = np.array(gt)
    
    for i in range(10):
        log_loss_ += log_loss(gt[:, i], pr[:, i])
        
    return log_loss_ / 10

In [11]:
log_loss_score(y_val, model.predict_proba(X_val))

0.3855734217281258

# Создание файла отправки

В файле с ответами должны быть вероятности для каждого класса. Сумма вероятностей в каждой строке может быть больше 1.

In [12]:
submission_columns = ['text_id'] + list(train.columns[2:-1])
submission = pd.concat([test['text_id'], pd.DataFrame(model.predict_proba(pd.DataFrame(test['text']))[:, :10])], axis=1)
submission.columns = submission_columns

In [13]:
submission.head()

,text_id,эймериоз,абсцесс,диспепсия молодняка,остертагиоз,мастит,инфекционный ринотрахеит,отёк вымени,тенденит,сибирская язва,лишай
0,294,0.158399,0.114065,0.121807,0.091488,0.087121,0.068731,0.047912,0.061100,0.096797,0.066527
1,295,0.649903,0.130241,0.424610,0.385480,0.143775,0.102555,0.068045,0.271119,0.140103,0.101376
2,296,0.497556,0.128907,0.417635,0.362003,0.085173,0.082718,0.053324,0.173252,0.108915,0.068368
3,297,0.092075,0.166353,0.087311,0.084409,0.066975,0.075544,0.051250,0.067557,0.096998,0.066321
4,298,0.099724,0.138278,0.103211,0.094123,0.186553,0.081351,0.088730,0.081809,0.114271,0.104003


In [14]:
submission_json = {str(k): {"span": list(), "label": list(v.values())} \
                   for k,v in submission.set_index('text_id').to_dict('index').items()}

submission_json['294']

{'span': [],
 'label': [0.15839938785217741,
  0.11406474371106279,
  0.12180665939855911,
  0.09148752585229543,
  0.08712122855132899,
  0.06873100350701257,
  0.0479117525232059,
  0.06110032039638013,
  0.0967968198778296,
  0.0665265225286165]}

In [15]:
with open('sample_submission.json', 'w') as final_submit:
    json.dump(submission_json, final_submit, indent=4)